In [2]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Step 2: Load Dataset
df = pd.read_csv('bengaluru_house_prices.csv')

# Step 3: Basic Exploration
print(df.shape)
print(df.columns)
print(df.head())
print(df.info())
print(df.isnull().sum())
print("Duplicates:", df.duplicated().sum())

# Step 4: Data Cleaning
# Drop unhelpful columns
df = df.drop(['area_type', 'society', 'balcony', 'availability'], axis=1)

# Drop rows with missing values
df = df.dropna()

# Step 5: Feature Engineering
# Convert 'size' to number of bedrooms (BHK)
df['bhk'] = df['size'].apply(lambda x: int(x.split(' ')[0]))

# Clean 'total_sqft' column
def convert_sqft(x):
    try:
        return float(x)
    except:
        tokens = x.split('-')
        if len(tokens) == 2:
            return (float(tokens[0]) + float(tokens[1])) / 2
        return None

df['total_sqft'] = df['total_sqft'].apply(convert_sqft)
df = df.dropna()

# Step 6: Create Price per Square Foot
df['price_per_sqft'] = df['price']*100000 / df['total_sqft']

# Step 7: Dimensionality Reduction (reduce number of locations)
df['location'] = df['location'].apply(lambda x: x.strip())
location_stats = df['location'].value_counts()
locations_less_than_10 = location_stats[location_stats <= 10]
df['location'] = df['location'].apply(lambda x: 'other' if x in locations_less_than_10 else x)

# Step 8: Outlier Removal (optional - based on business understanding)
df = df[df['total_sqft']/df['bhk'] >= 300]

# Step 9: One-Hot Encoding
dummies = pd.get_dummies(df['location'], drop_first=True)
df = pd.concat([df.drop('location', axis=1), dummies], axis=1)

# Step 10: Features and Target
X = df.drop(['price', 'size', 'price_per_sqft'], axis=1)
y = df['price']

# Step 11: Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 12: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 13: Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Step 14: Model Evaluation
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

# Step 15: Deploy with Gradio
!pip install gradio --quiet

import gradio as gr

# Store columns to match new input later
model_features = X.columns

# Gradio Prediction Function
def predict_price(location, sqft, bath, bhk):
    # Encode location
    loc_data = np.zeros(len(model_features))
    if location in model_features:
        loc_index = list(model_features).index(location)
        loc_data[loc_index] = 1

    # Fill other inputs
    loc_data[list(model_features).index('total_sqft')] = sqft
    loc_data[list(model_features).index('bath')] = bath
    loc_data[list(model_features).index('bhk')] = bhk

    # Scale and predict
    loc_data_scaled = scaler.transform([loc_data])
    pred = model.predict(loc_data_scaled)[0]
    return round(pred, 2)


    # Create input UI
# locations = sorted(df['location'].unique()) # This line is causing the error.
locations = [loc for loc in model_features if loc not in ['total_sqft', 'bath', 'bhk']] # locations should be picked up from model_features
# locations should contain the dummy variable location names, which were generated during one-hot encoding.

gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Dropdown(locations, label="Location"),
        gr.Number(label="Total Square Feet"),
        gr.Number(label="Number of Bathrooms"),
        gr.Number(label="Number of Bedrooms (BHK)")
    ],
    outputs=gr.Number(label="Predicted Price (Lakh ₹)"),
    title="🏡 Bengaluru House Price Predictor",
    description="Enter the property details to get an estimated selling price."
).launch()

(13320, 9)
Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')
              area_type   availability                  location       size  \
0  Super built-up  Area         19-Dec  Electronic City Phase II      2 BHK   
1            Plot  Area  Ready To Move          Chikka Tirupathi  4 Bedroom   
2        Built-up  Area  Ready To Move               Uttarahalli      3 BHK   
3  Super built-up  Area  Ready To Move        Lingadheeranahalli      3 BHK   
4  Super built-up  Area  Ready To Move                  Kothanur      2 BHK   

   society total_sqft  bath  balcony   price  
0  Coomee        1056   2.0      1.0   39.07  
1  Theanmp       2600   5.0      3.0  120.00  
2      NaN       1440   2.0      3.0   62.00  
3  Soiewre       1521   3.0      1.0   95.00  
4      NaN       1200   2.0      1.0   51.00  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (tot

<ipython-input-2-52e2b2548216>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft'] = df['price']*100000 / df['total_sqft']
<ipython-input-2-52e2b2548216>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location'] = df['location'].apply(lambda x: x.strip())
<ipython-input-2-52e2b2548216>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

MSE: 11479.502591547123
R² Score: 0.5303123010113314
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af94599638159b009e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
